In [29]:
import time
from itertools import product
import numpy as np
from scipy.stats import poisson, skellam
import matplotlib.pyplot as plt

maxNumCounted = 25
creditRate = 10
transfer_cost = 2
expectedRent1 = 3
expectedReturn1 = 3
expectedRent2 = 4
expectedReturn2 = 2
maxCarsAvail = 20
maxCarsTransferred = 5

In [30]:
def iterateInplace(value, getStatesIter, getActionsIter, getStateTransitionsIter, 
                 gamma=1.0, max_iter=None, Theta=np.finfo(np.float32).resolution):
    it = 1
    new_value = value
    while True:
        delta = 0.0
        print("Iteration {}".format(it), end="", flush=True)
        for state in getStatesIter():
            print(str(state)+"\r", end="", flush=True)
            v = value[state]
            qsa = [np.sum([prob*(reward + gamma*value[next_state])
                           for next_state, prob, reward 
                           in getStateTransitionsIter(state,action)]
                         )
                   for action in getActionsIter(state)]
            new_value[state] = max(qsa)
            delta = max(delta, abs(v-new_value[state]))
        print("[Done] delta={}".format(delta))
        if max_iter and it >= max_iter or delta < Theta: break
        it += 1
    return new_value

In [69]:
def evalDetermPolicy(value, policy, getStatesIter, getStateTransitionsIter, gamma, max_iter=None, Theta=1.0e-5):
    it = 1
    while True:
        delta = 0
        print("Iteration {}  ".format(it), end="", flush=True)
        for state in getStatesIter():
            v = value[state]
            action = policy[state]
            value[state] = np.sum([prob*(reward + gamma*value[next_state])
                           for next_state, prob, reward 
                           in getStateTransitionsIter(state,action)])
            delta = max(delta, abs(v-value[state]))
            print(".", end="", flush=True)
        print("\n[Done] delta={}".format(delta))
        if max_iter and it >= max_iter or delta < Theta: break
        it += 1
    return value

def evalDetermPolicy_2(value, policy, getStatesIter, getStateTransitionsSum, gamma, max_iter=None, Theta=1.0e-5):
    it = 1
    while True:
        delta = 0
        print("Iteration {}  ".format(it), end="", flush=True)
        for state in getStatesIter():
            v = value[state]
            action = policy[state]
            value[state] = np.sum([w_reward + prob_sum*gamma*value[next_state]
                           for next_state, prob_sum, w_reward
                           in getStateTransitionsSum(state,action)])
            delta = max(delta, abs(v-value[state]))
            print(".", end="", flush=True)
        print("\n[Done] delta={}".format(delta))
        if max_iter and it >= max_iter or delta < Theta: break
        it += 1
    return value


In [70]:
poissonCache = np.array([[poisson.pmf(x,expect)
                          for x in range(maxNumCounted+1)]
                         for expect in range(expectedReturn2,expectedRent2+1)])

def getPoisson(n,e):
    return poissonCache[e-expectedReturn2, n]

returned_prob = np.array([[getPoisson(returned1, expectedReturn1) * 
                        getPoisson(returned2, expectedReturn2)
                        for returned2 in range(maxNumCounted+1)] 
                        for returned1 in range(maxNumCounted+1)]
                        )
rented_prob = np.array([[getPoisson(rented1, expectedRent1) * 
                        getPoisson(rented2, expectedRent2)
                        for rented2 in range(maxNumCounted+1)]
                        for rented1 in range(maxNumCounted+1)]
                      )

def getJacksCarTransitions(cars_left, transfer):
    cars1, cars2 = cars_left
    if cars1 < transfer or cars2 < -transfer:
        return
    cars_avail_1 = cars1-transfer
    cars_avail_2 = cars2+transfer
    
    for returned1,returned2 in product(range(maxNumCounted+1), range(maxNumCounted+1)):
        c1 = min(cars_avail_1+returned1, maxCarsAvail)
        c2 = min(cars_avail_2+returned2, maxCarsAvail)
        for rented1,rented2 in product(range(maxNumCounted+1), range(maxNumCounted+1)):
            left_1 = max(0,c1-rented1)
            left_2 = max(0,c2-rented2)
            yield ((max(0,c1-rented1), max(0,c2-rented2)),
                returned_prob[returned1,returned2] * rented_prob[rented1,rented2],
                creditRate*(c1+c2 - (left_1+left_2)) - transfer_cost*abs(transfer)
                  )

In [27]:
l = [p for s,p,r in getJacksCarTransitions((1,1),0)]
len(l), np.sum(l)

(456976, 0.9999999999997595)

In [76]:
value = np.zeros((maxCarsAvail+1,maxCarsAvail+1), dtype=np.float64)
policy = np.zeros((maxCarsAvail+1,maxCarsAvail+1), dtype=np.int8)

In [ ]:
v = evalDetermPolicy_2(value,
               policy,
               lambda : product(range(maxCarsAvail+1),range(maxCarsAvail+1)),
               cache_3, #getJacksCarTransitions_3,
               gamma=0.9,
               max_iter=40,
               Theta=0.01
              )

In [ ]:
iterateInplace(value, 
               lambda : product(range(maxCarsAvail+1),range(maxCarsAvail+1)),
               lambda s: range(-s[1], s[0]+1),
               getJacksCarTransitions,
               gamma=0.9,
               max_iter=1
              )

In [34]:
def getJacksCarTransitions_2(cars_left, transfer):
    cars1, cars2 = cars_left
    if cars1 < transfer or cars2 < -transfer:
        return
    cars_avail_1 = cars1-transfer
    cars_avail_2 = cars2+transfer
    
    for returned1 in range(maxCarsAvail-cars_avail_1+1):
        ret1_prob = getPoisson(returned1,expectedReturn1)
        if returned1 == maxCarsAvail-cars_avail_1:
            ret1_prob += np.sum([getPoisson(r1,expectedReturn1) 
                                 for r1 in range(returned1+1, maxNumCounted+1)])
            
        for returned2 in range(maxCarsAvail-cars_avail_2+1):
            ret2_prob = getPoisson(returned2,expectedReturn2)
            if returned2 == maxCarsAvail-cars_avail_2:
                ret2_prob += np.sum([getPoisson(r2,expectedReturn2) 
                                     for r2 in range(returned2+1, maxNumCounted+1)])

            c1 = cars_avail_1+returned1
            c2 = cars_avail_2+returned2
            for rented1,rented2 in product(range(maxNumCounted+1), range(maxNumCounted+1)):
                left_1 = max(0,c1-rented1)
                left_2 = max(0,c2-rented2)
                yield ((max(0,c1-rented1), max(0,c2-rented2)),
                    ret1_prob * ret2_prob * rented_prob[rented1,rented2],
                    creditRate*(c1+c2 - (left_1+left_2)) - transfer_cost*abs(transfer)
                      )

In [12]:
def getJacksCarTransitions_3(cars_left, transfer):
    cars1, cars2 = cars_left
    if cars1 < transfer or cars2 < -transfer:
        return
    cars_avail_1 = cars1-transfer
    cars_avail_2 = cars2+transfer
    
    for returned1 in range(maxCarsAvail-cars_avail_1+1):
        ret1_prob = getPoisson(returned1,expectedReturn1)
        if returned1 == maxCarsAvail-cars_avail_1:
            ret1_prob += np.sum([getPoisson(r1,expectedReturn1) 
                                 for r1 in range(returned1+1, maxNumCounted+1)])
            
        for returned2 in range(maxCarsAvail-cars_avail_2+1):
            ret2_prob = getPoisson(returned2,expectedReturn2)
            if returned2 == maxCarsAvail-cars_avail_2:
                ret2_prob += np.sum([getPoisson(r2,expectedReturn2)
                                     for r2 in range(returned2+1, maxNumCounted+1)])

            c1 = cars_avail_1+returned1
            c2 = cars_avail_2+returned2
            for rented1 in range(c1+1):
                rent1_prob = getPoisson(rented1, expectedRent1)
                if (rented1 == c1):
                    rent1_prob += np.sum([getPoisson(r1,expectedRent1)
                                 for r1 in range(rented1+1, maxNumCounted+1)])
                    
                for rented2 in range(c2+1):
                    rent2_prob = getPoisson(rented2, expectedRent2)
                    if (rented2 == c2):
                        rent2_prob += np.sum([getPoisson(r2,expectedRent2)
                                     for r2 in range(rented2+1, maxNumCounted+1)])
                    
                    left_1 = c1-rented1
                    left_2 = c2-rented2
                    yield ((left_1, left_2),
                        ret1_prob * ret2_prob * rent1_prob * rent2_prob,
                        creditRate*(rented1+rented2) - transfer_cost*abs(transfer)
                          )

In [74]:
def compileDistribution(state, action, getStateTransitionsIter):
    summary = np.zeros(shape=(maxCarsAvail+1,maxCarsAvail+1,2), dtype=np.float64)
    for next_state, prob, reward in getStateTransitionsIter(state,action):
        summary[next_state] += [prob, prob*reward]
        
    # normalize weighted reward by probability of next state
    #for s in summary:
    #    s[1] /= s[0]
        
    return summary

class getJacksCarCachedTransitionsSummary:
    
    def __init__(self, getStateTransitionsIter):
        self.store = {}
        self.get_trans = getStateTransitionsIter
        
    def __call__(self, state, action):
        key = state,action
        if not key in self.store:
            summary_dist = compileDistribution(state, action, self.get_trans)
            self.store[key] = summary_dist
        else:
            summary_dist = self.store[key]
            
        return [(enum,)+tuple(summary_dist[enum]) for enum in np.ndindex(summary_dist.shape[:-1])]

cache_3 = getJacksCarCachedTransitionsSummary(getJacksCarTransitions_3)
cache = getJacksCarCachedTransitionsSummary(getJacksCarTransitions)

In [ ]:
cache_3((1,1),0)

In [44]:
value_3 = np.zeros((maxCarsAvail+1,maxCarsAvail+1), dtype=np.float64)

In [ ]:
v = iterateInplace(value,
               lambda : product(range(maxCarsAvail+1),range(maxCarsAvail+1)),
               lambda s: range(-s[1], s[0]+1),
               getJacksCarTransitions_3,
               gamma=0.9,
               max_iter=4
              )

In [42]:
l = [p for s,p,r in getJacksCarTransitions_3((1,1),1)]
len(l), np.sum(l)

(52668, 0.9999999999997595)

In [80]:
np.min(v)

451.88009257681296